Data *Online Retail*

**Oleh:** 

Dea Restika Augustina Pratiwi (06211740000023)

##### Download Data : [online_retail.csv](https://drive.google.com/file/d/1sqrmytAewju0K6fnUoiQhp_GK2Dy__nz/view)

In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

In [3]:
data = pd.read_csv('D:/online_retail.csv')
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/11 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/11 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/11 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/11 12:50,4.15,12680.0,France


In [4]:
data.StockCode.nunique()

4070

In [5]:
data.Description.nunique()

4223

In [6]:
data.groupby("StockCode").Description.nunique().sort_values(ascending=False).iloc[0:10]

StockCode
20713     8
23084     7
21830     6
85175     6
85172     5
72807A    5
23343     5
21181     5
23131     5
23196     4
Name: Description, dtype: int64

In [7]:
data.loc[data.StockCode == "23244"].Description.value_counts()

ROUND STORAGE TIN VINTAGE LEAF    108
STORAGE TIN VINTAGE LEAF            7
CANNISTER VINTAGE LEAF DESIGN       2
Name: Description, dtype: int64

In [8]:
missing_percentage = data.isnull().sum() / data.shape[0] * 100
missing_percentage

InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64

In [9]:
data[data.Description.isnull()].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/10 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/10 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/10 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/10 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/10 14:34,0.0,NaN,United Kingdom


In [10]:
data[data.Description.isnull()].CustomerID.isnull().value_counts()

True    1454
Name: CustomerID, dtype: int64

In [11]:
data[data.Description.isnull()].UnitPrice.value_counts()

0.0    1454
Name: UnitPrice, dtype: int64

In [12]:
data[data.CustomerID.isnull()].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/10 11:52,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/10 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/10 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/10 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/10 14:32,1.66,NaN,United Kingdom


In [13]:
data.loc[data.CustomerID.isnull(), ["UnitPrice", "Quantity"]].describe()

,UnitPrice,Quantity
count,135080.000000,135080.000000
mean,8.076577,1.995573
std,151.900816,66.696153
min,-11062.060000,-9600.000000
25%,1.630000,1.000000
50%,3.290000,1.000000
75%,5.450000,3.000000
max,17836.460000,5568.000000


In [14]:
data.loc[data.Description.isnull()==False, "lowercase_descriptions"] = data.loc[
    data.Description.isnull()==False,"Description"
].apply(lambda l: l.lower())

data.lowercase_descriptions.dropna().apply(
    lambda l: np.where("nan" in l, True, False)
).value_counts()

False    539724
True        731
Name: lowercase_descriptions, dtype: int64

In [15]:
data.loc[data.lowercase_descriptions.isnull()==False, "lowercase_descriptions"] = data.loc[
    data.lowercase_descriptions.isnull()==False, "lowercase_descriptions"
].apply(lambda l: np.where("nan" in l, None, l))

In [16]:
data = data.loc[(data.CustomerID.isnull()==False) & (data.lowercase_descriptions.isnull()==False)].copy()
data.isnull().sum().sum()

0

In [17]:
data["IsCancelled"]=np.where(data.InvoiceNo.apply(lambda l: l[0]=="C"), True, False)
data.IsCancelled.value_counts() / data.shape[0] * 100

False    97.81007
True      2.18993
Name: IsCancelled, dtype: float64

In [18]:
data.loc[data.IsCancelled==True].describe()

,Quantity,UnitPrice,CustomerID
count,8896.000000,8896.000000,8896.000000
mean,-30.882981,18.862815,14991.575202
std,1170.746458,444.590459,1707.208018
min,-80995.000000,0.010000,12346.000000
25%,-6.000000,1.450000,13506.000000
50%,-2.000000,2.950000,14895.000000
75%,-1.000000,4.950000,16393.000000
max,-1.000000,38970.000000,18282.000000


In [19]:
data = data.loc[data.IsCancelled==False].copy()
data = data.drop("IsCancelled", axis=1)

In [20]:
data['Description'] = data['Description'].str.strip()

In [21]:
basket1 = data.groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack()
basket1.shape

(18535, 3864)

In [22]:
basket1 = basket1.applymap(lambda x: 1 if x>0 else 0)
basket1

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [23]:
basket1.iloc[0].value_counts()

0    3857
1       7
Name: 536365, dtype: int64

In [31]:
itemsets1 = apriori(basket1, min_support=0.03, use_colnames=True)

In [32]:
itemsets1.sort_values('support',ascending=False)

,support,itemsets
86,0.106339,(WHITE HANGING HEART T-LIGHT HOLDER)
69,0.091934,(REGENCY CAKESTAND 3 TIER)
35,0.086323,(JUMBO BAG RED RETROSPOT)
58,0.074454,(PARTY BUNTING)
5,0.074184,(ASSORTED COLOUR BIRD ORNAMENT)
...,...,...
20,0.030861,(HAND WARMER OWL DESIGN)
21,0.030861,(HANGING HEART JAR T-LIGHT HOLDER)
18,0.030753,(GUMBALL COAT RACK)
55,0.030645,(PAPER BUNTING RETROSPOT)


In [33]:
rules1 = association_rules(itemsets1, metric="lift", min_threshold=1)
rules1.shape

(0, 9)

In [34]:
basket2 = (data[data['Country'] =="France"].groupby(['InvoiceNo', 'Description'])
           ['Quantity'].sum().unstack().reset_index().set_index('InvoiceNo'))

In [35]:
basket2 = basket2.applymap(lambda x: 1 if x>0 else 0)
basket2

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
itemsets2 = apriori(basket2, min_support = 0.1, use_colnames=True)
itemsets2.shape

(47, 2)

In [37]:
itemsets2.sort_values('support',ascending=False)

,support,itemsets
12,0.771208,(POSTAGE)
13,0.187661,(RABBIT NIGHT LIGHT)
15,0.179949,(RED TOADSTOOL LED NIGHT LIGHT)
11,0.172237,(PLASTERS IN TIN WOODLAND ANIMALS)
9,0.169666,(PLASTERS IN TIN CIRCUS PARADE)
33,0.167095,"(POSTAGE, RABBIT NIGHT LIGHT)"
18,0.159383,(ROUND SNACK BOXES SET OF4 WOODLAND)
35,0.159383,"(RED TOADSTOOL LED NIGHT LIGHT, POSTAGE)"
3,0.154242,(LUNCH BAG RED RETROSPOT)
37,0.149100,"(POSTAGE, ROUND SNACK BOXES SET OF4 WOODLAND)"


In [38]:
rules2 = association_rules(itemsets2, metric="lift", min_threshold=1)
rules2.shape

(54, 9)

In [39]:
rules2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(POSTAGE),(LUNCH BAG APPLE DESIGN),0.771208,0.125964,0.105398,0.136667,1.084966,0.008254,1.012397
1,(LUNCH BAG APPLE DESIGN),(POSTAGE),0.125964,0.771208,0.105398,0.836735,1.084966,0.008254,1.401350
2,(LUNCH BAG RED RETROSPOT),(POSTAGE),0.154242,0.771208,0.123393,0.800000,1.037333,0.004441,1.143959
3,(POSTAGE),(LUNCH BAG RED RETROSPOT),0.771208,0.154242,0.123393,0.160000,1.037333,0.004441,1.006855
4,(POSTAGE),(LUNCH BAG WOODLAND),0.771208,0.118252,0.102828,0.133333,1.127536,0.011631,1.017402
5,(LUNCH BAG WOODLAND),(POSTAGE),0.118252,0.771208,0.102828,0.869565,1.127536,0.011631,1.754070
6,(LUNCH BOX WITH CUTLERY RETROSPOT),(POSTAGE),0.143959,0.771208,0.115681,0.803571,1.041964,0.004659,1.164758
7,(POSTAGE),(LUNCH BOX WITH CUTLERY RETROSPOT),0.771208,0.143959,0.115681,0.150000,1.041964,0.004659,1.007107
8,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.169666,0.172237,0.102828,0.606061,3.518770,0.073605,2.101246
9,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.172237,0.169666,0.102828,0.597015,3.518770,0.073605,2.060459


In [40]:
rulesc2 = association_rules(itemsets2, metric = "confidence", min_threshold = 0.5)
rulesc2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(LUNCH BAG APPLE DESIGN),(POSTAGE),0.125964,0.771208,0.105398,0.836735,1.084966,0.008254,1.401350
1,(LUNCH BAG RED RETROSPOT),(POSTAGE),0.154242,0.771208,0.123393,0.800000,1.037333,0.004441,1.143959
2,(LUNCH BAG WOODLAND),(POSTAGE),0.118252,0.771208,0.102828,0.869565,1.127536,0.011631,1.754070
3,(LUNCH BOX WITH CUTLERY RETROSPOT),(POSTAGE),0.143959,0.771208,0.115681,0.803571,1.041964,0.004659,1.164758
4,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.169666,0.172237,0.102828,0.606061,3.518770,0.073605,2.101246
5,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.172237,0.169666,0.102828,0.597015,3.518770,0.073605,2.060459
6,(PLASTERS IN TIN CIRCUS PARADE),(POSTAGE),0.169666,0.771208,0.149100,0.878788,1.139495,0.018253,1.887532
7,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN SPACEBOY),0.172237,0.136247,0.105398,0.611940,4.491411,0.081932,2.225826
8,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.136247,0.172237,0.105398,0.773585,4.491411,0.081932,3.655955
9,(PLASTERS IN TIN SPACEBOY),(POSTAGE),0.136247,0.771208,0.115681,0.849057,1.100943,0.010607,1.515746


In [41]:
ruless = rules2[(rules2['confidence'] >= 0.8 )&(rules2['lift'] >= 1.1 )]

In [42]:
ruless

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(LUNCH BAG WOODLAND),(POSTAGE),0.118252,0.771208,0.102828,0.869565,1.127536,0.011631,1.754070
11,(PLASTERS IN TIN CIRCUS PARADE),(POSTAGE),0.169666,0.771208,0.149100,0.878788,1.139495,0.018253,1.887532
15,(PLASTERS IN TIN SPACEBOY),(POSTAGE),0.136247,0.771208,0.115681,0.849057,1.100943,0.010607,1.515746
19,(RABBIT NIGHT LIGHT),(POSTAGE),0.187661,0.771208,0.167095,0.890411,1.154566,0.022370,2.087725
22,(RED TOADSTOOL LED NIGHT LIGHT),(POSTAGE),0.179949,0.771208,0.159383,0.885714,1.148476,0.020605,2.001928
27,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.159383,0.771208,0.149100,0.935484,1.213011,0.026183,3.546272
31,(SET/6 RED SPOTTY PAPER CUPS),(POSTAGE),0.138817,0.771208,0.118252,0.851852,1.104568,0.011195,1.544344
35,(STRAWBERRY LUNCH BOX WITH CUTLERY),(POSTAGE),0.123393,0.771208,0.115681,0.937500,1.215625,0.020519,3.660668
39,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.128535,0.133676,0.102828,0.800000,5.984615,0.085646,4.331620
40,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563


In [43]:
ruless.shape

(17, 9)

In [45]:
ruless.to_excel (r'D:/output.xlsx', index = False, header=True)

## ANALISIS 15 NEGARA

Ingin difilter 15 negara dari data yang sudah di preprocessig. Maka, filter dilakukan denan menggunakan bantuan excel. Sebelumnya dilakukan export data ke excel.

In [ ]:
data.to_excel(r'D:/filter.xlsx', index = False, header = True)

In [46]:
data15 = pd.read_excel('D:/15negara.xlsx')
data15

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,lowercase_descriptions
0,536389,22941,CHRISTMAS LIGHTS 10 REINDEER,6,12/1/10 10:03,8.50,12431,Australia,christmas lights 10 reindeer
1,536389,21622,VINTAGE UNION JACK CUSHION COVER,8,12/1/10 10:03,4.95,12431,Australia,vintage union jack cushion cover
2,536389,21791,VINTAGE HEADS AND TAILS CARD GAME,12,12/1/10 10:03,1.25,12431,Australia,vintage heads and tails card game
3,536389,35004C,SET OF 3 COLOURED FLYING DUCKS,6,12/1/10 10:03,5.45,12431,Australia,set of 3 coloured flying ducks
4,536389,35004G,SET OF 3 GOLD FLYING DUCKS,4,12/1/10 10:03,6.35,12431,Australia,set of 3 gold flying ducks
...,...,...,...,...,...,...,...,...,...
16548,581578,22993,SET OF 4 PANTRY JELLY MOULDS,12,12/9/11 12:16,1.25,12713,Germany,set of 4 pantry jelly moulds
16549,581578,22907,PACK OF 20 NAPKINS PANTRY DESIGN,12,12/9/11 12:16,0.85,12713,Germany,pack of 20 napkins pantry design
16550,581578,22908,PACK OF 20 NAPKINS RED APPLES,12,12/9/11 12:16,0.85,12713,Germany,pack of 20 napkins red apples
16551,581578,23215,JINGLE BELL HEART ANTIQUE SILVER,12,12/9/11 12:16,2.08,12713,Germany,jingle bell heart antique silver


In [47]:
np.sum(data15.isnull())

InvoiceNo                 0
StockCode                 0
Description               0
Quantity                  0
InvoiceDate               0
UnitPrice                 0
CustomerID                0
Country                   0
lowercase_descriptions    0
dtype: int64

In [48]:
data15.Country.nunique()

15

In [49]:
basket3 = data15.groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack()
basket3.shape

(799, 2149)

In [50]:
basket3 = basket3.applymap(lambda x: 1 if x>0 else 0)
basket3

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,...,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536389,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581493,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
itemsets3 = apriori(basket3, min_support=0.1, use_colnames=True)

In [52]:
itemsets3.sort_values('support',ascending=False)

,support,itemsets
2,0.689612,(POSTAGE)
5,0.215269,(ROUND SNACK BOXES SET OF4 WOODLAND)
8,0.188986,"(POSTAGE, ROUND SNACK BOXES SET OF4 WOODLAND)"
4,0.132666,(ROUND SNACK BOXES SET OF 4 FRUITS)
3,0.130163,(REGENCY CAKESTAND 3 TIER)
7,0.122653,"(ROUND SNACK BOXES SET OF 4 FRUITS, POSTAGE)"
1,0.116395,(PLASTERS IN TIN WOODLAND ANIMALS)
6,0.113892,(SPACEBOY LUNCH BOX)
9,0.108886,"(ROUND SNACK BOXES SET OF 4 FRUITS, ROUND SNAC..."
0,0.107635,(PLASTERS IN TIN CIRCUS PARADE)


In [53]:
rules3 = association_rules(itemsets3, metric="lift", min_threshold=1)
rules3.shape

(12, 9)

In [54]:
rules3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ROUND SNACK BOXES SET OF 4 FRUITS),(POSTAGE),0.132666,0.689612,0.122653,0.924528,1.340650,0.031165,4.112641
1,(POSTAGE),(ROUND SNACK BOXES SET OF 4 FRUITS),0.689612,0.132666,0.122653,0.177858,1.340650,0.031165,1.054969
2,(POSTAGE),(ROUND SNACK BOXES SET OF4 WOODLAND),0.689612,0.215269,0.188986,0.274047,1.273045,0.040534,1.080967
3,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.215269,0.689612,0.188986,0.877907,1.273045,0.040534,2.542225
4,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.132666,0.215269,0.108886,0.820755,3.812692,0.080327,4.377972
5,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.215269,0.132666,0.108886,0.505814,3.812692,0.080327,1.755076
6,"(ROUND SNACK BOXES SET OF 4 FRUITS, POSTAGE)",(ROUND SNACK BOXES SET OF4 WOODLAND),0.122653,0.215269,0.101377,0.826531,3.839523,0.074973,4.523743
7,"(ROUND SNACK BOXES SET OF 4 FRUITS, ROUND SNAC...",(POSTAGE),0.108886,0.689612,0.101377,0.931034,1.350084,0.026288,4.500626
8,"(POSTAGE, ROUND SNACK BOXES SET OF4 WOODLAND)",(ROUND SNACK BOXES SET OF 4 FRUITS),0.188986,0.132666,0.101377,0.536424,4.043421,0.076305,1.870964
9,(ROUND SNACK BOXES SET OF 4 FRUITS),"(POSTAGE, ROUND SNACK BOXES SET OF4 WOODLAND)",0.132666,0.188986,0.101377,0.764151,4.043421,0.076305,3.438698


In [55]:
rulesc3 = association_rules(itemsets3, metric = "confidence", min_threshold = 0.5)
rulesc3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ROUND SNACK BOXES SET OF 4 FRUITS),(POSTAGE),0.132666,0.689612,0.122653,0.924528,1.340650,0.031165,4.112641
1,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.215269,0.689612,0.188986,0.877907,1.273045,0.040534,2.542225
2,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.132666,0.215269,0.108886,0.820755,3.812692,0.080327,4.377972
3,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.215269,0.132666,0.108886,0.505814,3.812692,0.080327,1.755076
4,"(ROUND SNACK BOXES SET OF 4 FRUITS, POSTAGE)",(ROUND SNACK BOXES SET OF4 WOODLAND),0.122653,0.215269,0.101377,0.826531,3.839523,0.074973,4.523743
5,"(ROUND SNACK BOXES SET OF 4 FRUITS, ROUND SNAC...",(POSTAGE),0.108886,0.689612,0.101377,0.931034,1.350084,0.026288,4.500626
6,"(POSTAGE, ROUND SNACK BOXES SET OF4 WOODLAND)",(ROUND SNACK BOXES SET OF 4 FRUITS),0.188986,0.132666,0.101377,0.536424,4.043421,0.076305,1.870964
7,(ROUND SNACK BOXES SET OF 4 FRUITS),"(POSTAGE, ROUND SNACK BOXES SET OF4 WOODLAND)",0.132666,0.188986,0.101377,0.764151,4.043421,0.076305,3.438698


In [57]:
rulest3 = rules3[(rules3['confidence'] >= 0.5 )&(rules3['lift'] >= 1.1 )]

In [58]:
rulest3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ROUND SNACK BOXES SET OF 4 FRUITS),(POSTAGE),0.132666,0.689612,0.122653,0.924528,1.340650,0.031165,4.112641
3,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.215269,0.689612,0.188986,0.877907,1.273045,0.040534,2.542225
4,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.132666,0.215269,0.108886,0.820755,3.812692,0.080327,4.377972
5,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.215269,0.132666,0.108886,0.505814,3.812692,0.080327,1.755076
6,"(ROUND SNACK BOXES SET OF 4 FRUITS, POSTAGE)",(ROUND SNACK BOXES SET OF4 WOODLAND),0.122653,0.215269,0.101377,0.826531,3.839523,0.074973,4.523743
7,"(ROUND SNACK BOXES SET OF 4 FRUITS, ROUND SNAC...",(POSTAGE),0.108886,0.689612,0.101377,0.931034,1.350084,0.026288,4.500626
8,"(POSTAGE, ROUND SNACK BOXES SET OF4 WOODLAND)",(ROUND SNACK BOXES SET OF 4 FRUITS),0.188986,0.132666,0.101377,0.536424,4.043421,0.076305,1.870964
9,(ROUND SNACK BOXES SET OF 4 FRUITS),"(POSTAGE, ROUND SNACK BOXES SET OF4 WOODLAND)",0.132666,0.188986,0.101377,0.764151,4.043421,0.076305,3.438698


In [59]:
rulest3.to_excel (r'D:/output2.xlsx', index = False, header=True)